<img src="https://cdn.comet.ml/img/notebook_logo.png">

[Comet](https://www.comet.com/site/products/ml-experiment-tracking/?utm_campaign=pytorch&utm_medium=colab) is an MLOps Platform that is designed to help Data Scientists and Teams build better models faster! Comet provides tooling to track, Explain, Manage, and Monitor your models in a single place! It works with Jupyter Notebooks and Scripts and most importantly it's 100% free to get started!

[PyTorch](https://pytorch.org/) is a popular open source machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.

PyTorch enables fast, flexible experimentation and efficient production through a user-friendly front-end, distributed training, and ecosystem of tools and libraries.

Instrument PyTorch with Comet to start managing experiments, create dataset versions and track hyperparameters for faster and easier reproducibility and collaboration.

[Find more information about our integration with Pytorch](https://www.comet.ml/docs/v2/integrations/ml-frameworks/pytorch/)

Curious about how Comet can help you build better models, faster? Find out more about [Comet](https://www.comet.com/site/products/ml-experiment-tracking/?utm_campaign=pytorch&utm_medium=colab) and our [other integrations](https://www.comet.ml/docs/v2/integrations/overview/)

Get a preview for what's to come. Check out a completed experiment created from this notebook [here](https://www.comet.com/examples/comet-example-pytorch-histogram-logging/).


# Install Dependencies

In [ ]:
%pip install comet_ml torch torchvision tqdm

# Initialize Comet

In [ ]:
import comet_ml

comet_ml.init(project_name="comet-example-pytorch-histogram-logging")

# Import Dependencies

In [ ]:
import torch
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch import nn, optim

from collections import OrderedDict
from tqdm import tqdm

# Create Comet Experiment

In [ ]:
experiment = comet_ml.Experiment()

# Define parameters

In [ ]:
hyper_params = {"batch_size": 100, "num_epochs": 3, "learning_rate": 0.01}
experiment.log_parameters(hyper_params)

# Load data

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

trainset = datasets.MNIST("/tmp", download=True, train=True, transform=transform)
valset = datasets.MNIST("/tmp", download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=hyper_params["batch_size"], shuffle=True
)
valloader = torch.utils.data.DataLoader(
    valset, batch_size=hyper_params["batch_size"], shuffle=True
)

input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Define Model and Optimizer

In [ ]:
model = nn.Sequential(
    OrderedDict(
        [
            ("linear0", nn.Linear(input_size, hidden_sizes[0])),
            ("activ0", nn.ReLU()),
            ("linear1", nn.Linear(hidden_sizes[0], hidden_sizes[1])),
            ("activ1", nn.ReLU()),
            ("linear2", nn.Linear(hidden_sizes[1], output_size)),
            ("output", nn.LogSoftmax(dim=1)),
        ]
    )
)
optimizer = optim.Adam(model.parameters(), lr=hyper_params["learning_rate"])

# Define helper functions to logs gradients and weights

In [ ]:
def to_numpy(x):
    return x.detach().numpy()


def update_gradient_map(model, gradmap):
    for name, layer in zip(model._modules, model.children()):
        if "activ" in name:
            continue

        if not hasattr(layer, "weight"):
            continue

        wname = "%s/%s.%s" % ("gradient", name, "weight")
        bname = "%s/%s.%s" % ("gradient", name, "bias")

        gradmap.setdefault(wname, 0)
        gradmap.setdefault(bname, 0)

        gradmap[wname] += layer.weight.grad
        gradmap[bname] += layer.bias.grad

    return gradmap


def log_gradients(gradmap, step):
    for k, v in gradmap.items():
        experiment.log_histogram_3d(to_numpy(v), name=k, step=step)


def log_weights(model, step):
    for name, layer in zip(model._modules, model.children()):
        if "activ" in name:
            continue

        if not hasattr(layer, "weight"):
            continue

        wname = "%s.%s" % (name, "weight")
        bname = "%s.%s" % (name, "bias")

        experiment.log_histogram_3d(to_numpy(layer.weight), name=wname, step=step)
        experiment.log_histogram_3d(to_numpy(layer.bias), name=bname, step=step)

# Train a Model

In [ ]:
def train(model, dataloader, epoch):
    gradmap = {}
    steps_per_epoch = len(dataloader.dataset) // hyper_params["batch_size"]
    total_loss = 0

    with experiment.train():
        model.train()
        for batch_idx, (data, target) in tqdm(enumerate(dataloader)):
            data = data.view(data.size(0), -1)
            optimizer.zero_grad()
            pred = model(data)

            loss = F.nll_loss(pred, target)
            loss.backward()

            gradmap = update_gradient_map(model, gradmap)
            optimizer.step()

            total_loss += loss.item()

        # scale gradients
        for k, v in gradmap.items():
            gradmap[k] = v / steps_per_epoch

        log_gradients(gradmap, epoch * steps_per_epoch)
        log_weights(model, epoch * steps_per_epoch)
        total_loss /= len(dataloader.dataset)

    experiment.log_metric("loss", total_loss, epoch=epoch)

In [ ]:
max_epochs = hyper_params["num_epochs"]
for epoch in range(max_epochs + 1):
    train(model, trainloader, epoch)

experiment.end()